In [139]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

 

import fix_yahoo_finance as yf
from pandas_datareader import data as pdr
df = pdr.get_data_yahoo('JPM', '2010-01-01', '2021-02-01')
print(len(df))
df = df.dropna() 
df = df.iloc[:,:4] 
print(len(df))
df.head()
df.info()

2789
2789
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2789 entries, 2010-01-04 to 2021-02-01
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   High    2789 non-null   float64
 1   Low     2789 non-null   float64
 2   Open    2789 non-null   float64
 3   Close   2789 non-null   float64
dtypes: float64(4)
memory usage: 108.9 KB


In [140]:
n=13
 
def RSI(df,n):
    "function to calculate RSI"
    df = df.copy()
    df['delta']=df['Close'] - df['Close'].shift(1)
    df['gain']=np.where(df['delta']>=0,df['delta'],0)
    df['loss']=np.where(df['delta']<0,abs(df['delta']),0)
    avg_gain = []
    avg_loss = []
    gain = df['gain'].tolist() #transforming the column into a numpy list
    loss = df['loss'].tolist()
    for i in range(len(df)):  #loop
        if i < n:
            avg_gain.append(np.NaN)
            avg_loss.append(np.NaN)
        elif i == n:
            avg_gain.append(df['gain'].rolling(n).mean().tolist()[n])
            avg_loss.append(df['loss'].rolling(n).mean().tolist()[n])
        elif i > n:
            avg_gain.append(((n-1)*avg_gain[i-1] + gain[i])/n)
            avg_loss.append(((n-1)*avg_loss[i-1] + loss[i])/n)
    df['avg_gain']=np.array(avg_gain)
    df['avg_loss']=np.array(avg_loss)
    df['RS'] = df['avg_gain']/df['avg_loss']
    df['RSI'] = 100 - (100/(1+df['RS']))
    return df['RSI']

df['RSI'] = RSI(df, n)

In [141]:
a = 13
b = 24
c = 9
 
def MACD(df,a,b,c):
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    df = df.copy()
    df["MA_Fast"]=df["Close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["Close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return df
 
df = MACD(df,a,b,c)

In [142]:
X= df[['RSI','MACD','Signal']]
df['up_down'] = np.where (df['Close'].shift(-1) > df['Close'],1,-1)

In [154]:


Y=df['up_down']
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)




from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
 

knn =KNeighborsClassifier(n_neighbors=28)
 

fitted_model = knn.fit(X_train, Y_train)
predictions = fitted_model.predict(X_test)
 


 

accuracy_train = accuracy_score(Y_train, knn.predict(X_train))
accuracy_test = accuracy_score(Y_test, knn.predict(X_test))
print("KNN")
print("Total Sum: ",len(X_test))
c_matrix=confusion_matrix(Y_test, predictions)
print("Sum of correct values= ", c_matrix[0][0]+c_matrix[1][1])

print("Accuracy: ",accuracy_test)
print("Confusion matrix:  ")
print(confusion_matrix(Y_test, predictions))


KNN
Total Sum:  911
Sum of correct values=  469
Accuracy:  0.5148188803512623
Confusion matrix:  
[[255 179]
 [263 214]]
